[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/FBI223/TOMOGRAFIA_UCZENIE_GLEBOKICH/blob/main/TASK2/task2.ipynb)



# Zadanie obowiazkowe : Odpalenie modelu i wizualizacja dzialania modelu

W tej części student samodzielnie **użyje gotowego, wytrenowanego modelu U-Net** do przeprowadzenia **segmentacji tomografii komputerowej płuc (CT)**.

Celem jest:

1. Wczytanie modelu z pliku `.ckpt`,
2. Wczytanie danych 3D konkretnego pacjenta (`.nii.gz`),
3. Przeprowadzenie predykcji maski nowotworu dla każdej warstwy CT,
4. Stworzenie animacji pokazującej segmentację,

Podpowiedz : Uzyj funkcji


In [ ]:
from onedrivedownloader import download
url = "https://ujchmura-my.sharepoint.com/:u:/g/personal/marcin_sztukowski_student_uj_edu_pl/EeIxSRDsbaROj51n9jl7TDoBTifXVohnxCVymwhyiGrBSQ?e=IRznML"
download(url, filename="epoch=29-step=53759.ckpt", unzip=False)

In [ ]:
import torch
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
from celluloid import Camera
from IPython.display import HTML, Video
import numpy as np
import warnings
from matplotlib.animation import FuncAnimation
import os


def load_trained_model(checkpoint_path, model_class = TumorSegmentation, device=None):
    """
    Wczytuje wytrenowany model PyTorch Lightning z pliku .ckpt

    Args:
        checkpoint_path: ścieżka do pliku checkpoint (.ckpt)
        model_class: klasa modelu (np. TumorSegmentation)
        device: 'cuda', 'cpu' lub None (automatycznie wykrywane)

    Returns:
        model: załadowany i gotowy do ewaluacji model
    """

    # automatyczne wykrycie urządzenia
    if device is None:
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # wczytanie modelu z checkpointu
    print(f"🔄 Loading model from: {checkpoint_path}")
    model = model_class.load_from_checkpoint(checkpoint_path)

    # przeniesienie na GPU/CPU
    model = model.to(device)
    model.eval()

    print(f"✅ Model loaded successfully on {device}")
    return model





def create_segmentation_video(scan, segmentation, save_path="/content/ct_tumor_animation.mp4",
                              step=2, fps=10):
    """
    Tworzy wideo MP4 z segmentacji CT bez użycia matplotlib.animation (działa w Colabie bez błędów).
    """
    os.makedirs("/content/tmp_frames", exist_ok=True)
    tmp_dir = "/content/tmp_frames"

    # --- zapis pojedynczych klatek ---
    for idx, i in enumerate(tqdm(range(0, len(scan), step), desc="Rendering frames")):
        plt.figure(figsize=(6, 6))
        plt.imshow(scan[i], cmap="bone")
        mask = np.ma.masked_where(segmentation[i] == 0, segmentation[i])
        plt.imshow(mask, alpha=0.5, cmap="autumn")
        plt.axis("off")
        frame_path = f"{tmp_dir}/frame_{idx:04d}.png"
        plt.savefig(frame_path, bbox_inches="tight", pad_inches=0)
        plt.close()

    # --- składanie filmu przez ffmpeg ---
    print("💾 Rendering wideo przez ffmpeg...")
    cmd = f"ffmpeg -y -framerate {fps} -i {tmp_dir}/frame_%04d.png -c:v libx264 -pix_fmt yuv420p {save_path}"
    os.system(cmd)

    # --- czyszczenie ---
    for f in os.listdir(tmp_dir):
        os.remove(os.path.join(tmp_dir, f))
    os.rmdir(tmp_dir)

    print(f"✅ Zapisano: {save_path}")




def predict_single_patient(model, ct_volume, device="cuda", threshold=0.5):
    """
    Segmentacja jednego pacjenta (CT volume) przy użyciu modelu (np. U-Net).

    Args:
        model: wytrenowany model PyTorch (np. U-Net)
        ct_volume: numpy array 3D (H, W, N) — pełny tomogram pacjenta
        device: 'cuda' lub 'cpu'
        threshold: próg binarny dla maski (np. 0.5)

    Returns:
        scan: lista 2D obrazów (oryginalnych slice'ów)
        segmentation: lista 2D masek binarnych
        preds: lista masek z wartościami sigmoid (float)
    """

    model.eval()
    scan = []
    segmentation = []
    preds = []

    for i in tqdm(range(ct_volume.shape[-1])):
        # --- przygotowanie slice’a ---
        slice_ = cv2.resize(ct_volume[:, :, i], (256, 256))
        scan.append(slice_)

        input_t = torch.tensor(slice_).unsqueeze(0).unsqueeze(0).float().to(device)

        # --- predykcja modelu ---
        with torch.no_grad():
            pred = torch.sigmoid(model(input_t))[0, 0].cpu().numpy()

        preds.append(pred)
        segmentation.append((pred > threshold).astype(np.uint8))

    scan = np.array(scan)
    segmentation = np.array(segmentation)
    preds = np.array(preds)

    return scan, segmentation, preds




In [ ]:



# ======================================================
# 👩‍🎓 CZĘŚĆ STUDENTA
# ======================================================



import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from celluloid import Camera
from IPython.display import Video
from tqdm import tqdm

import nibabel as nib



THRESHOLD = 0.5
MAX_VAL = 3071
CROP_SIZE = 30
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

#stale do stworzenia wideo
STEPS = 2
FPS=10


# wczytaj pobrany model / checkpoint
checkpoint = "/content/POBRANY_MODEL..." # TODO
model = load_trained_model( ) #TODO
subject = Path("/content/Task06_Lung/imagesTs/DANE_PACJENTA.nii.gz") #TODO




# Uzyj nib.load oraz get_fdata aby wczytac dane dowolnego pacjenta
# pamietaj aby podzielic przez MAX_VAL wczytane dane aby ustandaryzowac dane
# pamietaj aby zrobic CROP wczytanych danych

subject = Path("/content/Task06_Lung/imagesTs/WYBIERZ_ZDJECIE") #TODO
ct = nib.load  #TODO load
ct = ct[...]  # TODO crop
scan, segmentation, preds =  ...  #TODO uzyj funkcji wyzej

#uzyj gotowej funkcji do stworzenia video z klatek   #TODO
Video("/content/VIDEO.mp4", embed=True) #TODO




In [ ]:



# ======================================================
# 👨‍🏫 CZĘŚĆ NAUCZYCIELA
# ======================================================


THRESHOLD = 0.5
MAX_VAL = 3071
CROP_SIZE = 30
#stale do stworzenia wideo
STEPS = 2
FPS=10


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
checkpoint = "/content/epoch=29-step=53759.ckpt"
model = load_trained_model(checkpoint_path=checkpoint, model_class=TumorSegmentation)
subject = Path("/content/Task06_Lung/imagesTs/lung_035.nii.gz")
ct = nib.load(subject).get_fdata() / MAX_VAL  # standardize
ct = ct[:,:,CROP_SIZE:]  # crop
scan, segmentation, preds = predict_single_patient(model, ct, device=DEVICE)
create_segmentation_video(scan, segmentation, "/content/ct_tumor_animation_35.mp4", step=STEPS, fps=FPS)
Video("/content/ct_tumor_animation_35.mp4", embed=True)


